## Einlesen der Daten und Definition Chatbot

In [4]:
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from bs4 import BeautifulSoup

# === Inhalte.txt laden ===
docs = []
chunks = []
with open("Inhalte.txt", "r", encoding="utf-8") as f:
    content = f.read()

blocks = content.split("============================================================")
for block in blocks:
    block = block.strip()
    if len(block) > 100:
        docs.append(Document(page_content=block))

print(f"📄 Datei 'Inhalte.txt' geladen. Module erkannt: {len(docs)}")

# === In kleinere Chunks aufteilen (wenn nötig) ===
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=60)

# manual chunking
for doc in docs:
    first_line = doc.page_content.split("\n")[0]  # e.g., "Modul: Betriebssystemeinführung"
    vorlesung = first_line.split(":", 1)[1].strip().upper()
    for chunk_text in splitter.split_text(doc.page_content):
        # Prepend the first line to each chunk and append it as well
        full_chunk = f"{vorlesung}\n{chunk_text}\n{vorlesung}"
        chunks.append(Document(
            page_content=full_chunk,
            metadata={"modul": vorlesung.lower()}
        ))

# === Modulbook-pi2 laden ===
xml_files = ["moduldb-pi2.xml", "modulbook_pi2_de.xml"]
docs = []

for xml_file in xml_files:
    with open(xml_file, "r", encoding="ISO-8859-1") as f:
        xml_content = f.read()

    soup = BeautifulSoup(xml_content, "xml")
    modules = soup.find_all("moduleheader")

    for module in modules:
        title = module.find("title").text.strip() if module.find("title") else "Kein Titel"
        cid = module.find("cid").text.strip() if module.find("cid") else "Kein Kürzel"
        cp = module.find("cp").text.strip() if module.find("cp") else "?"
        convenor = module.find("convenor").text.strip() if module.find("convenor") else "?"

        types = module.find_all("type")
        ctypes = ", ".join(t.text for t in types) if types else "Unbekannt"

        content = f"""
Modul: {title}
Kürzel: {cid}
Leistungspunkte (ECTS): {cp}
Verantwortlich: {convenor}
Veranstaltungstyp(en): {ctypes}
"""
        docs.append(Document(page_content=content.strip()))

print(f"{len(docs)} Dokumente aus xml Dateien extrahiert.")
chunks.extend(splitter.split_documents(docs))


# === Vektordatenbank mit Embeddings aufbauen ===
embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1") #  sentence-transformers/all-mpnet-base-v2
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 7})

# === LLM konfigurieren ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",  # ggf. deine Adresse
    model="llama3-70b",  # oder mistral-7b je nach ollama setup
    temperature=0.3
)

# === Prompt definieren ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Modulbeschreibungen:\n"
        "{context}\n\n"
        "Beantworte die folgende Frage **nur auf Basis dieser Informationen**. "
        "Wenn die Antwort nicht im Text steht, sage: 'Nicht enthalten'.\n\n"
        "Frage: {question}\n\n"
        "Antwort:"
    )
)

# === Retrieval-QA-Kette starten ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

📄 Datei 'Inhalte.txt' geladen. Module erkannt: 3
121 Dokumente aus xml Dateien extrahiert.


## Run Chatbot

In [3]:
# === Chat starten ===
print("\n🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.invoke(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print("⚠️ Fehler:", e)



🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: {'query': 'Welche Literatur benötige ich für Betriebssystemeinführung?', 'result': 'Für das Modul "Betriebssystemeinführung" werden folgende Literaturquellen empfohlen:\n- Powers, Peek, O’Reilly, Loukides: *Unix Power Tools*, O’Reilly, 2002\n- Rosenblatt: *Learning the Korn Shell*, O’Reilly, 1995\n- Stapelberg: *UNIX SYSTEM V.4*, Addison-Wesley, 1995\n- Patrick Ditchen: *Shell-Skript Programmierung*, mitp, 2003\n- Christian Meißer: *Bash – Arbeiten und programmieren mit der Shell*, open source PRESS, 2011'}
🤖 Bot: Auf Wiedersehen!


## Test

In [2]:
retriever.invoke("Literatur Betriebssystemeinführung?")

[Document(metadata={'modul': 'betriebswirtschaftslehre'}, page_content='BETRIEBSWIRTSCHAFTSLEHRE\nLiteratur:\n- Jean-Paul Thommen, Ann-Kristin Achleitner: *Allgemeine Betriebswirtschaftslehre*, Springer\n- Thomas Straub: *Einführung in die Allgemeine Betriebswirtschaftslehre*, Pearson\n- Henner Schierenbeck, Claudia Wöhle: *Grundzüge der Betriebswirtschaftslehre*, Oldenbourg\n- Han\nBETRIEBSWIRTSCHAFTSLEHRE'),
 Document(metadata={'modul': 'betriebssystemeinführung'}, page_content='BETRIEBSSYSTEMEINFÜHRUNG\nLiteratur:\n- Powers, Peek, O’Reilly, Loukides: *Unix Power Tools*, O’Reilly, 2002\n- Rosenblatt: *Learning the Korn Shell*, O’Reilly, 1995\n- Stapelberg: *UNIX SYSTEM V.4*, Addison-Wesley, 1995\n- Patrick Ditchen: *Shell-Skript Programmierung*, mitp, 2003\n- Christian Meißer: *Bash – Arbeiten und programmieren mit der Shell*, open source PRESS, 2011\n\nAngeboten in folgenden Semestern:\nWS 2024/25, WS 2023/24, WS 2022/23, WS 2021/22, WS 2020/21\nBETRIEBSSYSTEMEINFÜHRUNG'),
 Document